In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [13:23<00:00, 18.24it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
362858,Laugh Now Cry Later (feat. Lil Durk),25,2021-02-12,Drake,https://open.spotify.com/track/2SAqBLGA283SUiw...,United States,top200,SAME_POSITION,503574.000,"10 May 2021, 16:37","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",Hip-Hop,rap,rnb,hip hop,Canadian,Hip-Hop,rap,rnb,trap,pop rap,spotify:track:2SAqBLGA283SUiwJ3xOUVI,0.761,0.518,0.000,-8.871,1.000,0.134,0.244,0.000,0.107,0.522,133.976,audio_features,2SAqBLGA283SUiwJ3xOUVI,https://api.spotify.com/v1/tracks/2SAqBLGA283S...,https://api.spotify.com/v1/audio-analysis/2SAq...,261493.000,4.000,2,459,hip-hop,hip-hop
160371,God's Plan,200,2019-09-04,Drake,https://open.spotify.com/track/6DCZcSspjsKoFjz...,United States,top200,NEW_ENTRY,217846.000,"16 Feb 2018, 20:15","25 Mar 2006, 06:09","Aubrey Drake Graham (born October 24, 1986) is...","Aubrey Drake Graham (born October 24, 1986) is...",Hip-Hop,rap,rnb,hip hop,Canadian,Hip-Hop,rap,2018,Canadian,2010s,spotify:track:6DCZcSspjsKoFjzjrWoCdn,0.754,0.449,7.000,-9.211,1.000,0.109,0.033,0.000,0.552,0.357,77.169,audio_features,6DCZcSspjsKoFjzjrWoCdn,https://api.spotify.com/v1/tracks/6DCZcSspjsKo...,https://api.spotify.com/v1/audio-analysis/6DCZ...,198973.000,4.000,15,453,hip-hop,hip-hop
249042,Heather,185,2020-07-17,Conan Gray,https://open.spotify.com/track/4xqrdfXkTW4T0Ra...,United States,top200,MOVE_DOWN,240442.000,"09 Apr 2021, 17:41","10 Oct 2018, 20:11",Conan Gray is an American singer-songwriter an...,Conan Gray is an American singer-songwriter an...,indie,pop,indie pop,american,youtube,pop,indie,sad,folk,indie pop,spotify:track:4xqrdfXkTW4T0RauPLv3WA,0.357,0.425,5.000,-7.301,1.000,0.033,0.584,0.000,0.322,0.270,102.078,audio_features,4xqrdfXkTW4T0RauPLv3WA,https://api.spotify.com/v1/tracks/4xqrdfXkTW4T...,https://api.spotify.com/v1/audio-analysis/4xqr...,198040.000,3.000,7,614,indie,indie
87759,Hold Me Down,44,2017-09-03,Daniel Caesar,https://open.spotify.com/track/1Mza2sr6tPhy6jj...,United States,viral50,MOVE_DOWN,NaN,"27 Feb 2022, 21:36","01 May 2016, 12:25",Daniel Caesar is Canadian singer and songwrite...,Daniel Caesar is Canadian singer and songwrite...,rnb,soul,Canadian,Neo-Soul,canada,rnb,sexy,calm,canada,quiet,spotify:track:1Mza2sr6tPhy6jjI3HB9fW,0.463,0.427,8.000,-9.799,0.000,0.055,0.628,0.001,0.091,0.231,116.059,audio_features,1Mza2sr6tPhy6jjI3HB9fW,https://api.spotify.com/v1/tracks/1Mza2sr6tPhy...,https://api.spotify.com/v1/audio-analysis/1Mza...,231268.000,4.000,31,7,rnb,rnb
26926,You & Me,199,2017-02-02,"Marc E. Bassy, G-Eazy",https://open.spotify.com/track/4QdovcM8j50QPly...,United States,top200,MOVE_DOWN,156785.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Marc+E.+Ba...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:7yCqehT9dKACTFy7YugP0J,0.449,0.635,1.000,-8.301,1.000,0.275,0.041,0.000,0.188,0.608,84.414,audio_features,7yCqehT9dKACTFy7YugP0J,https://api.spotify.com/v1/tracks/7yCqehT9dKAC...,https://api.spotify.com/v1/audio-analysis/7yCq...,218493.000,4.000,133,31,NaN,other


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.395
female       0.136
group        0.136
non-binary   0.017
Name: gender, dtype: float64

In [28]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender
5044,I Don’t Wanna Live Forever (Fifty Shades Darke...,182,2018-01-06,"ZAYN, Taylor Swift",https://open.spotify.com/track/3NdDpSvN911VPGi...,United States,top200,MOVE_UP,180315.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Zayn,+Tayl...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,367,NaN,other,NaN
150884,Speechless,119,2018-11-16,Dan + Shay,https://open.spotify.com/track/3GJ4hzg4lrGwU51...,United States,top200,MOVE_DOWN,297305.000,NaN,"12 Nov 2013, 22:04",Dan + Shay is an American country music duo co...,Dan + Shay is an American country music duo co...,country,pop,country pop,american,modern country,NaN,NaN,NaN,NaN,NaN,spotify:track:3GJ4hzg4lrGwU51Y3VARbF,0.616,0.438,1.000,-5.968,1.000,0.030,0.356,0.000,0.240,0.386,135.929,audio_features,3GJ4hzg4lrGwU51Y3VARbF,https://api.spotify.com/v1/tracks/3GJ4hzg4lrGw...,https://api.spotify.com/v1/audio-analysis/3GJ4...,213387.000,4.000,50,664,country,country,group
453531,#BrooklynBloodPop!,2,2021-05-04,SyKo,https://open.spotify.com/track/7K9Z3yFNNLv5kwT...,United States,viral50,SAME_POSITION,NaN,"19 May 2021, 06:54","26 Feb 2007, 02:12","hyperpop star <a href=""https://www.last.fm/mus...","hyperpop star <a href=""https://www.last.fm/mus...",scenecore,hyperpop,bop,american,music,hyperpop,tiktokcore,alternative,crunkcore,Vibe,spotify:track:7K9Z3yFNNLv5kwTjQYGjnu,0.691,0.814,1.000,-3.788,0.000,0.117,0.016,0.000,0.366,0.509,132.012,audio_features,7K9Z3yFNNLv5kwTjQYGjnu,https://api.spotify.com/v1/tracks/7K9Z3yFNNLv5...,https://api.spotify.com/v1/audio-analysis/7K9Z...,145611.000,4.000,1,58,scenecore,other,NaN
222185,Ayala (Outro),79,2017-08-27,XXXTENTACION,https://open.spotify.com/track/5xbrhx2tMMcN68I...,United States,top200,MOVE_DOWN,307443.000,"07 May 2021, 17:31","18 Dec 2016, 00:43","Jahseh Dwayne Onfroy (January 23, 1998 – June ...","Jahseh Dwayne Onfroy (January 23, 1998 – June ...",Hip-Hop,rap,cloud rap,trap,emo rap,Hip-Hop,hip hop,emo,rap,rnb,spotify:track:5xbrhx2tMMcN68IT3cJbhS,0.570,0.285,9.000,-14.125,0.000,0.038,0.737,0.013,0.108,0.097,83.485,audio_features,5xbrhx2tMMcN68IT3cJbhS,https://api.spotify.com/v1/tracks/5xbrhx2tMMcN...,https://api.spotify.com/v1/audio-analysis/5xbr...,99707.000,4.000,75,15,hip-hop,hip-hop,NaN
133924,SLOW DANCING IN THE DARK,181,2018-12-31,Joji,https://open.spotify.com/track/0rKtyWc8bvkriBt...,United States,top200,MOVE_DOWN,214296.000,"18 Feb 2020, 06:08","04 Jul 2015, 13:40","George Miller, known by his stage name Joji, i...","George Miller, known by his stage name Joji, i...",Lo-Fi,rnb,Hip-Hop,japanese,alternative rnb,soul,rnb,alternative rnb,joji,Hip-Hop,spotify:track:0rKtyWc8bvkriBthvHKY8d,0.515,0.479,3.000,-7.458,1.000,0.026,0.544,0.006,0.191,0.284,88.964,audio_features,0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274.000,4.000,11,1139,lo-fi,lo-fi,NaN


Age from artist

-- taking the data from the summary

In [29]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d*, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

In [ ]:
df.sample()

# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df